![Neptune + Catalyst](https://neptune.ai/wp-content/uploads/2023/09/catalyst.svg)

# Neptune + Catalyst

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/catalyst/notebooks/Neptune_Catalyst.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/catalyst/notebooks/Neptune_Catalyst.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/catalyst-integration/runs/details?viewId=e4dd3103-0792-43bd-b5db-7d5124ef6c55&detailsTab=metadata&shortId=CATALYST-38823"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/catalyst/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Create a `NeptuneLogger`,
* Log a catalyst model using `NeptuneLogger`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
! pip install -U catalyst ipywidgets "neptune-client<1.0.0" tensorboard torchvision

## Import libraries

In [ ]:
import os
from collections import OrderedDict

from catalyst import dl
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch import nn, optim
from torch.utils.data import DataLoader

## Prepare hyper-parameters

In [ ]:
my_hparams = {"lr": 0.07, "batch_size": 32}

They will be logged automatically.

## Prepare model, criterion, optimizer and data loaders

Define components needed for the Catalyst Runner.

In [ ]:
model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), my_hparams["lr"])
loaders = OrderedDict(
    {
        "training": DataLoader(
            MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()),
            batch_size=my_hparams["batch_size"],
        ),
        "validation": DataLoader(
            MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()),
            batch_size=my_hparams["batch_size"],
        ),
    }
)

Notice two keys: `"training"` and `"validation"`. These names are used in Neptune as namespaces to organize your run.

## Create runner

In [ ]:
my_runner = dl.SupervisedRunner()

## Create NeptuneLogger

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/catalyst-integration](https://app.neptune.ai/common/catalyst-integration). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

neptune_logger = dl.NeptuneLogger(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. 

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
from catalyst import dl
import neptune  # Only needed to access the `ANONYMOUS_API_TOKEN` for anonymous logging

neptune_logger = dl.NeptuneLogger(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/catalyst-integration",
    tags=["notebook-example", "quickstart"],
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

For more ways to customize the logger behavior, see the [Catalyst integration guide](https://docs.neptune.ai/integrations/catalyst).

## Pass neptune_logger to the train method of the Runner

In [ ]:
my_runner.train(
    loggers={"neptune": neptune_logger},
    hparams=my_hparams,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=5,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk=[1]),
        dl.CheckpointCallback(
            logdir="checkpoints",
            loader_key="validation",
            metric_key="loss",
            minimize=True,
        ),
    ],
    valid_loader="validation",
    valid_metric="loss",
    minimize_valid_metric=True,
)

Few explanations:

1. `train` method executes model training, so here you start logging metadata to Neptune.
1. `neptune_logger` is passed to the `loggers` parameter.
1. Notice that `my_hparams` are also passed, to that they can be logged as well.

## Log best model

After training, use `log_artifact()` method to log model checkpoint.

In [ ]:
my_runner.log_artifact(
    path_to_artifact="./checkpoints/model.best.pth", tag="best_model", scope="experiment"
)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
neptune_logger.run.stop()

## Analyze logged metadata in the Neptune app

Go to the run link and explore metadata (metrics, hparams, model checkpoint) that were logged to the run in Neptune.

Link should look like this: https://app.neptune.ai/o/common/org/catalyst-integration/e/CATALYST-38823